# Lesson 6.5 - API Wrappers

## Lesson 1 key concepts

## Spotipy API

Create an Spotify account and follow these steps to register an app: https://developer.spotify.com/documentation/general/guides/app-settings/

After the app is created, you can see it on your dashboard
https://developer.spotify.com/dashboard/applications

Click on it and you'll find the client id and client secret.

In [ ]:
#!pip install spotipy

#### Authentification

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentials
#sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id='<your client id here>',
#                                                           client_secret='<your client secret here>'))

##### storing secrets

In [3]:
secrets_file = open("secrets.txt","r")

FileNotFoundError: [Errno 2] No such file or directory: 'secrets.txt'

In [ ]:
string = secrets_file.read()

In [ ]:
string

In [ ]:
string.split('\n')

In [ ]:
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()

In [ ]:
secrets_dict

#### authentication with secrets

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['cid'],
                                                           client_secret=secrets_dict['csecret']))

#### Searching songs with 'queries' with `sp.search`

In [ ]:
results = sp.search(q='Lady Gaga', limit=50)
results

## Lesson 2 key concepts

### Understand Previous Output:

In [ ]:
results.keys() # We can see that we only have a list (dictionary) of tracks

In [ ]:
results["tracks"].keys() # Let's check the values

In [ ]:
results["tracks"]["href"] # Query we have searched 

In [ ]:
results["tracks"]["items"] #items (actual tracks)

In [ ]:
len(results["tracks"]["items"])

In [ ]:
results["tracks"]["limit"]#Limit we have chosen

In [ ]:
results["tracks"]["next"] #link to the next 'page' (next 50 tracks)

In [ ]:
results["tracks"]["offset"] # Actual offset (starting point)

In [ ]:
results["tracks"]["previous"] #Previous search

In [ ]:
results["tracks"]["total"]

### Exploring the tracks:

In [ ]:
len(results["tracks"]["items"]) # 50 Tracks (as limited, it is the maximum)

In [ ]:
results["tracks"]["items"][0]# Explore the first song

In [ ]:
results["tracks"]["items"][0].keys() # We will focus on album, artists, id, name, popularity, type and uri

### Check Albums:

In [ ]:
# we have more info about the album
results["tracks"]["items"][0]["album"] 

In [ ]:
# Will check artists, id, name, release date, total tracks
results["tracks"]["items"][0]["album"].keys() 

In [ ]:
# check name of album
results["tracks"]["items"][0]["album"]["name"] 

In [ ]:
# List with artists and information
results["tracks"]["items"][0]["album"]["artists"] 

Spotify objects are identified by either a "url", a "uri" or an "id". 

- The `id` is an alphanumeric code, and it's the nuclear part of the identifier.

- The `uri` contains "spotify:track" before the id. An uri is useful because it can be searched manually in the Spotify app.

- The `url` is a link to the song on the Spotify web player.

In [ ]:
# Album ref
results["tracks"]["items"][0]["album"]["href"] 

In [ ]:
# Album ID
results["tracks"]["items"][0]["album"]["id"] 

In [ ]:
# Album uri
results["tracks"]["items"][0]["album"]["uri"] 

In [ ]:
#songs in the album
results["tracks"]["items"][0]["album"]["total_tracks"] 

### About the song itself:

In [ ]:
# Track name
results["tracks"]["items"][0]["name"] 

In [ ]:
# Track artists
results["tracks"]["items"][0]["artists"] 

In [ ]:
# Track ID
results["tracks"]["items"][0]["id"] 

In [ ]:
# Popularity index
results["tracks"]["items"][0]["popularity"] 

### Activity 1 - Searching multiple artists 

### Activity 2 - Exploring the tracks

## Function to get the name of the song + artists involved in track:

In [ ]:
#THIS FUNCTION WILL BE USED AGAIN!
def get_name_artists_from_track(track):
    return [(track["name"],artist["name"]) for artist in track["artists"]]

In [ ]:
my_track = results["tracks"]["items"][0]

In [ ]:
get_name_artists_from_track(my_track)

In [ ]:
[get_name_artists_from_track(list_el) for list_el in results["tracks"]["items"][0:5]]

In [ ]:
all_songs_artists = list(map(get_name_artists_from_track,results['tracks']['items']))
all_songs_artists

In [ ]:
#flattening is not in the standard library but used a lot. you'll thank me later
def flatten(input_list):
    return [item for sublist in input_list for item in sublist]

# outlist=[]
# for sublist in input_list:
#     for item in sublist:
#         outlist.append(item)
# return outlist

In [ ]:
flatten(all_songs_artists)

In [ ]:
def get_name_artists_from_list(tracks):
    return flatten(list(map(get_name_artists_from_track,tracks['items'])))

In [ ]:
get_name_artists_from_list(results['tracks'])

### Playlists

We will need to collect a "database" of songs. Playlists are a good way to access relatively large amounts of songs.

In [ ]:
# we will use our 'first paid music' playlist as an example:
playlist = sp.user_playlist_tracks("spotify", "3LgwHwHUnheGm88n8pDdA7")

In [ ]:
playlist

In [ ]:
# Let's look at items and total:
playlist.keys() 

In [ ]:
playlist["items"][0].keys()

In [ ]:
playlist["items"][0]["track"]

In [ ]:
playlist["items"][0]["track"]["name"]

In [ ]:
# 15 songs!! Let's check items:
playlist["total"] 

In [ ]:
 # It is limited to 100 tracks, we will have to fix it:
len(playlist["items"])

Activity: build a function that gets the names and artists of all songs in a playlist

In [ ]:
#first we need to get a list of individual tracks from the playlist
def get_name_artist_from_playlist_item(playlist_item):
    return get_name_artists_from_track(playlist_item['track'])

In [ ]:
get_name_artist_from_playlist_item(playlist["items"][0])

In [ ]:
def get_name_artists_from_playlist(input_playlist):
    return list(map(get_name_artist_from_playlist_item,input_playlist["items"]))

In [ ]:
get_name_artists_from_playlist(playlist)

In [ ]:
def get_name_artists_from_playlist(input_playlist):
    return flatten(list(map(get_name_artist_from_playlist_item,input_playlist["items"])))

In [ ]:
get_name_artists_from_playlist(playlist)

# Handling large playlists

In [ ]:
# we will need more songs for our clustering
#this is the playlist of previous cohorts
playlist = sp.user_playlist_tracks("spotify", "4rnleEAOdmFAbRcNCgZMpY")

In [ ]:
playlist["total"] 

In [ ]:
# It is limited to 100 tracks, we will have to fix it:
len(playlist["items"])

Function to extract all songs from a playlist

In [ ]:
playlist['next']

In [ ]:
sp.next(playlist)

In [ ]:
from random import randint
from time import sleep

def get_playlist_tracks(playlist_id):
    results = sp.user_playlist_tracks("spotify",playlist_id)
    tracks = results['items']
    while results['next']!=None:
        results = sp.next(results)
        tracks = tracks + results['items']
        sleep(randint(1,3))
    return tracks

In [ ]:
# this will take at least around num_songs_in_playlist * (avg_sleep_time + processing_time) = 53 * (2+0.1) = 110 seconds

all_tracks = get_playlist_tracks("4rnleEAOdmFAbRcNCgZMpY")
len(all_tracks)

### Audio features

You can check here an explanation of the audio features: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/

In [ ]:
# here we look only at the first page
playlist = sp.user_playlist_tracks("spotify", "4rnleEAOdmFAbRcNCgZMpY")

In [ ]:
playlist["items"][33]["track"]["name"]

In [ ]:
playlist["items"][46]["track"]["name"]

In [ ]:
# get the uri of a single song:
song_uri = playlist["items"][33]["track"]["uri"]
song_uri

In [ ]:
# get the audio features for that song
sp.audio_features(song_uri)

In [ ]:
# get the uri of a single song:
song_uri = playlist["items"][46]["track"]["uri"]

In [ ]:
# get the audio features for that song
sp.audio_features(song_uri)